In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from numpy import asarray
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import LSTM, Bidirectional, Dense
from sklearn.preprocessing import LabelBinarizer
from keras.utils.np_utils import to_categorical
%config Completer.use_jedi = False
import matplotlib.pyplot as plt
from tensorflow.keras import layers, Model, utils

In [2]:
df = pd.read_csv('../../data/anonymized_bsc_dataPVZ.csv')

/Users/clementinelacey/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (18,20,21,23,24,26,27,29,31,33) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df.head()

,Sttr Term,course_#,course_title,Sttr Student Load,Sttr Stu Credits,Sttr Attempted Cred,Sttr Stu Final Grades,Sttr Cmpl Cred,Sttr Term Gpa,faculty,...,FA5 Enr,FA5 Class,FA5 Major,FA6 Enr,FA6 Class,FA6 Major,Graduated,Grad Term,Grad Year,Grad Major
0,15/SP,DA 101 A,Basic Ballet,F,0.5,3.0,W,3.0,3.667,Melissa Turnage,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,18/SP,1718.0,Psychology
1,15/SP,HI 155 A-ES,Reforming America,F,1.0,NaN,A-,3.0,3.667,Victoria Ott,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,18/SP,1718.0,Psychology
2,15/SP,PL 200 A-CI,Ethical Choice,F,1.0,NaN,B+,3.0,3.667,William Myers,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,18/SP,1718.0,Psychology
3,15/SP,IDS 200 A-CI,Intro to Human Rights (GP),F,1.0,NaN,A,3.0,3.667,Sandra Sprayberry,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,18/SP,1718.0,Psychology
4,15/WI,GEN E299 49,Exploration Project,F,1.0,1.0,S,1.0,0.000,Vincent Gawronski,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,18/SP,1718.0,Psychology


# Preprocessing

In [4]:
#def preprocessing(data):
    #changing nomenclature for term so that we can sort in chronological order by that column
df['Sttr Term'] = df['Sttr Term'].replace({'WI':'A','SP':'B', 'SU':'C', 'FA':'D'}, regex=True)
    #dropping completely null rows
df.drop(df.index[51518:51537], inplace=True)
    #isolating students who would have been able to graduate in the timeframe of our given data
df = df.loc[(df['Cohort'] == '14/FA') | (df['Cohort'] == '15/FA') | (df['Cohort'] == '16/FA')
              | (df['Cohort'] == '17/FA')]
df = df.dropna(subset=['Sttr Term'])
df['alt_id'] = df['alt_id'].astype(int)
    #encoding our target
mapping = {'N': 0, 'Y': 1}
df['Graduated'] = df['Graduated'].map(mapping)
df = df.sort_values(['Sttr Term'], ascending=True)
df.head()

,Sttr Term,course_#,course_title,Sttr Student Load,Sttr Stu Credits,Sttr Attempted Cred,Sttr Stu Final Grades,Sttr Cmpl Cred,Sttr Term Gpa,faculty,...,FA5 Enr,FA5 Class,FA5 Major,FA6 Enr,FA6 Class,FA6 Major,Graduated,Grad Term,Grad Year,Grad Major
6784,14/D,EH 208 B,Intermediate Writing,F,1.00,NaN,F,1.00,1.110,Sandra Sprayberry,...,NaN,NaN,NaN,NaN,NaN,NaN,1,18/SP,1718.0,Urban Environmental Studies
7369,14/D,BI 115 A-SM,Organismal Biology,F,1.00,3.0,F,1.00,0.557,Andrew Gannon,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
7370,14/D,PY 101 F,Introduction to Psychology,F,1.00,NaN,C-,1.00,0.557,Gabrielle Smith,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
7371,14/D,EH 102 C,Sem Critical Thinking Writing,F,1.00,NaN,F,1.00,0.557,Steven Carter,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
1114,14/D,MS 120 D,Voice,F,0.25,NaN,A,3.25,3.591,Erin Ludwick,...,NaN,NaN,NaN,NaN,NaN,NaN,1,18/SP,1718.0,Biology


## EDA

In [5]:
#investigating how many classes students are taking to find outliers
df.alt_id.value_counts(ascending=False)

1139    64
1469    62
2046    62
1002    62
2328    60
        ..
2030     3
2516     3
1732     3
1191     3
1857     3
Name: alt_id, Length: 1455, dtype: int64

In [6]:
#Checking for Course Title Duplicates
print(df['course_title'].value_counts())

Exploration Project               3097
Sem Critical Thinking Writing     1210
Introduction to Psychology         973
Calculus I                         660
Introduction to Sociology          651
                                  ... 
Panorama of Spanish Literature       1
ED*499 Internship I                  1
Advanced Conversational Arabic       1
SPTP: Italian Cinema                 1
Black Studies Capstone               1
Name: course_title, Length: 952, dtype: int64


In [7]:
df['course_title'].nunique()

952

## One Hot Encoding Course Title ##

In [ ]:
    #data = df_4[['course_title']]
#encoder = OneHotEncoder(handle_unknown='ignore')
    #encoded = OneHotEncoder(sparse=False)
#encoder.fit(df)
    #encoded_df = encoded.fit_transform(data)
#df_encoded = encoder.transform(df)
    #encoded_df

In [ ]:
#ohe = OneHotEncoder()
#dummies = ohe.fit_transform(df_4[['course_title']])
#dummies_df = pd.DataFrame(dummies.todense(), columns=ohe.get_feature_names(), index=df_4.index)
#ohe_df = pd.concat([df_4, dummies_df], axis=1)

## Creating Target and Feature Variables ##

In [ ]:
#from sklearn.model_selection import train_test_split

#y = df['Graduated']
#X = df.drop(columns=['Graduated'], axis=1)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2021)

## Sorting Semester Term - Ascending ##

In [ ]:
df.query('course_title==0')

In [ ]:
df = df.sort_values(['Sttr Term'], ascending=True)

student_ids = df.alt_id.unique()

course_sequences = []

for s_id in student_ids:
    frame = df[df['alt_id'] == s_id]
    course_ids = ' '.join(frame.course_title.str.replace(' ', '').to_list())
    course_sequences.append(course_ids)

In [ ]:
#np.array(course_sequences)[1]

In [ ]:


#tokenizer = Tokenizer()

#tokenizer.fit_on_texts(course_sequences)

#encoded_sequences = tokenizer.texts_to_sequences(course_sequences)

In [ ]:


#padded_sequences = pad_sequences(
  # encoded_sequences, maxlen=None, dtype='int32', padding='post',
    #truncating='post', value=0.0

In [ ]:
#from sklearn.preprocessing import OneHotEncoder

#feature_encoder = OneHotEncoder(handle_unknown='ignore')

#feature_ids = list(tokenizer.word_index.values())

#feature_encoder.fit(np.array(feature_ids).reshape(-1,1))

#feature_encoder.transform(np.array(padded_sequences[0]).reshape(-1,1)).reshape(-1,1)

In [ ]:
#padded_sequences[0]

In [ ]:
#padded_reshaped = padded_sequences.reshape(1455, 80, 1)

In [ ]:


#model = Sequential()
#input_layer = Input(shape=(80, 1))
#sequential_1 = LSTM(64, activation='relu')

#output_layer = Dense(1, activation='sigmoid')

#model.add(input_layer)
#model.add(sequential_1)
#model.add(output_layer)

#model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])

In [ ]:
#model.fit(padded_reshaped, target, epochs=10)

In [ ]:
!pip install numpy==1.19.5

## Taking out course_title next to course names in column titles ##

In [ ]:
#def preprocessing_courses(data):
    #data.drop(data.index[51518:51537], inplace=True)
    #data = data.loc[(data['Cohort'] == '14/FA') | (data['Cohort'] == '15/FA') | (data['Cohort'] == '16/FA')
     #         | (data['Cohort'] == '17/FA')]
   # data = data.dropna(subset=['Sttr Term'])
   # mapping = {'N': 0, 'Y': 1}
   # data['Graduated'] = data['Graduated'].map(mapping)
   # y = data['Graduated']
   # X = data['course_title']
   # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2021)
   # X_train_final, X_val, y_train_final, y_val = train_test_split(X_train, y_train, test_size=1000, random_state=42)
   # data['alt_id'] = data['alt_id'].astype(int)
   # df = data.sort_values(['Sttr Term'], ascending=True)
    #return X

In [ ]:
target = []
for s_id in student_ids:
    grad = df[df['alt_id'] == s_id].iloc[0].Graduated
    target.append(grad)
    
def modeling (data):
    student_ids = data.alt_id.unique()
    course_sequences = []
    for s_id in student_ids:
        frame = data[data['alt_id'] == s_id]
        course_ids = ' '.join(frame.course_title.str.replace(' ', '').to_list())
        course_sequences.append(course_ids)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(course_sequences)
    encoded_sequences = tokenizer.texts_to_sequences(course_sequences)
    padded_sequences = pad_sequences(
    encoded_sequences, maxlen=None, dtype='int32', padding='post',
    truncating='post', value=0.0)
    feature_encoder = OneHotEncoder(handle_unknown='ignore')
    feature_ids = list(tokenizer.word_index.values())
    feature_encoder.fit(np.array(feature_ids).reshape(-1,1))
    feature_encoder.transform(np.array(padded_sequences[0]).reshape(-1,1)).reshape(-1,1)
    padded_reshaped = padded_sequences.reshape(1455, 45, 1)
    
    model = Sequential()
    input_layer = Input(shape=(80, 1))
    sequential_1 = LSTM(64, activation='relu')
    output_layer = Dense(1, activation='sigmoid')

    model.add(input_layer)
    model.add(sequential_1)
    model.add(output_layer)

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])
    model.fit(padded_reshaped, target, epochs=10)
    pd.DataFrame(feature_encoder.transform(np.array(padded_sequences[0]).reshape(-1,1)).todense())
    return np.array(padded_sequences[0]).reshape(-1,1)

### Baseline Course Sequence

In [23]:
target = []
for s_id in student_ids:
    grad = df[df['alt_id'] == s_id].iloc[0].Graduated
    target.append(grad)

In [24]:
#Baseline Course Sequence
student_ids = df.alt_id.unique()
course_sequences = []
for s_id in student_ids:
    frame = df[df['alt_id'] == s_id]
    course_ids = ' '.join(frame.course_title.str.replace(' ', '').to_list())
    course_sequences.append(course_ids)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(course_sequences)
encoded_sequences = tokenizer.texts_to_sequences(course_sequences)
padded_course_sequences = pad_sequences(encoded_sequences, maxlen=45, dtype='int32', padding='post',
truncating='post', value=0.0)
feature_encoder = OneHotEncoder(handle_unknown='ignore')
feature_ids = list(tokenizer.word_index.values())
feature_encoder.fit(np.array(feature_ids).reshape(-1,1))
feature_encoder.transform(np.array(padded_course_sequences[0]).reshape(-1,1)).reshape(-1,1)
padded_reshaped_courses = padded_course_sequences.reshape(1455, 45, 1)

model = Sequential()
input_layer = Input(shape=(80, 1))
sequential_1 = LSTM(64, activation='relu')
output_layer = Dense(1, activation='sigmoid')

model.add(input_layer)
model.add(sequential_1)
model.add(output_layer)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])
model.fit(padded_reshaped_courses, np.array(target), epochs=10, validation_split = .2)


pd.DataFrame(feature_encoder.transform(np.array(padded_course_sequences[0]).reshape(-1,1)).todense())
np.array(padded_course_sequences[0]).reshape(-1,1)

Epoch 1/10
37/37 [==============================] - 1s 19ms/step - loss: 27.1097 - accuracy: 0.3411 - val_loss: 17.4348 - val_accuracy: 0.4089
Epoch 2/10
37/37 [==============================] - 0s 13ms/step - loss: 15.8508 - accuracy: 0.4785 - val_loss: 10.5783 - val_accuracy: 0.4570
Epoch 3/10
37/37 [==============================] - 0s 12ms/step - loss: 11.8984 - accuracy: 0.5052 - val_loss: 7.2697 - val_accuracy: 0.5120
Epoch 4/10
37/37 [==============================] - 0s 12ms/step - loss: 6.7395 - accuracy: 0.5713 - val_loss: 7.8343 - val_accuracy: 0.5086
Epoch 5/10
37/37 [==============================] - 0s 12ms/step - loss: 7.3227 - accuracy: 0.6065 - val_loss: 9.3956 - val_accuracy: 0.5120
Epoch 6/10
37/37 [==============================] - 0s 12ms/step - loss: 13.5577 - accuracy: 0.5997 - val_loss: 12.4444 - val_accuracy: 0.6632
Epoch 7/10
37/37 [==============================] - 1s 14ms/step - loss: 14.0710 - accuracy: 0.8239 - val_loss: 9.0772 - val_accuracy: 0.7835
Epoch

array([[ 40],
       [ 13],
       [ 12],
       [ 10],
       [  1],
       [ 65],
       [191],
       [102],
       [  4],
       [ 10],
       [334],
       [134],
       [ 43],
       [ 40],
       [542],
       [424],
       [  6],
       [ 50],
       [341],
       [134],
       [323],
       [  3],
       [ 85],
       [  3],
       [  1],
       [413],
       [425],
       [  9],
       [543],
       [789],
       [ 22],
       [ 16],
       [ 47],
       [371],
       [485],
       [400],
       [401],
       [304],
       [624],
       [402],
       [527],
       [181],
       [790],
       [  0],
       [  0]], dtype=int32)

In [33]:
target = []
for s_id in student_ids:
    grad = df[df['alt_id'] == s_id].iloc[0].Graduated
    target.append(grad)

In [34]:
#baseline grades
student_ids = df.alt_id.unique()
grade_sequences = []
for s_id in student_ids:
    frame = df[df['alt_id'] == s_id]
    grades = ' '.join(frame['Sttr Stu Final Grades'].to_list())
    grade_sequences.append(grades)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(grade_sequences)
encoded_sequences = tokenizer.texts_to_sequences(grade_sequences)
padded_grade_sequences = pad_sequences(
encoded_sequences, maxlen=45, dtype='int32', padding='post',
truncating='post', value=0.0)
feature_encoder = OneHotEncoder(handle_unknown='ignore')
feature_ids = list(tokenizer.word_index.values())
feature_encoder.fit(np.array(feature_ids).reshape(-1,1))
feature_encoder.transform(np.array(padded_grade_sequences[0]).reshape(-1,1)).reshape(-1,1)
padded_reshaped_grades = padded_grade_sequences.reshape(1455, 45, 1)

model = Sequential()
input_layer = Input(shape=(45, 1))
sequential_1 = LSTM(64, activation='relu')
output_layer = Dense(1, activation='sigmoid')

model.add(input_layer)
model.add(sequential_1)
model.add(output_layer)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])
model.fit(padded_reshaped_grades, np.array(target), epochs=10, validation_split = .2)


#pd.DataFrame(feature_encoder.transform(np.array(padded_grade_sequences[0]).reshape(-1,1)).todense())
#np.array(padded_grade_sequences[0]).reshape(-1,1)

Epoch 1/10
37/37 [==============================] - 1s 16ms/step - loss: 0.5903 - accuracy: 0.6641 - val_loss: 0.6327 - val_accuracy: 0.5601
Epoch 2/10
37/37 [==============================] - 0s 12ms/step - loss: 0.4227 - accuracy: 0.7612 - val_loss: 0.5662 - val_accuracy: 0.8522
Epoch 3/10
37/37 [==============================] - 0s 12ms/step - loss: 0.3575 - accuracy: 0.9210 - val_loss: 0.3403 - val_accuracy: 0.9072
Epoch 4/10
37/37 [==============================] - 0s 12ms/step - loss: 0.3724 - accuracy: 0.9012 - val_loss: 0.4378 - val_accuracy: 0.8591
Epoch 5/10
37/37 [==============================] - 0s 12ms/step - loss: 0.3681 - accuracy: 0.9364 - val_loss: 0.4368 - val_accuracy: 0.8729
Epoch 6/10
37/37 [==============================] - 0s 12ms/step - loss: 0.2708 - accuracy: 0.9330 - val_loss: 0.3238 - val_accuracy: 0.8797
Epoch 7/10
37/37 [==============================] - 0s 13ms/step - loss: 0.1763 - accuracy: 0.9476 - val_loss: 0.2415 - val_accuracy: 0.9107
Epoch 8/10
37

In [35]:
padded_course_sequences[100]

array([ 72,  10,   2,  65,   1,   5,   4, 288,   8,  14,  82,   1,  60,
         3,  29,  23, 101,  52, 206,   6, 110,   1,  71, 272, 360,  97,
       672, 673, 590,  67, 243, 442, 135, 136, 137, 197, 516,  30, 244,
       197, 286, 499, 500,  86,   0], dtype=int32)

In [36]:
padded_grade_sequences[100]

array([2, 4, 2, 2, 2, 4, 2, 3, 4, 3, 2, 1, 2, 6, 3, 1, 2, 1, 1, 5, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 3, 4, 2, 3, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0], dtype=int32)

In [37]:
hooray = np.stack((padded_grade_sequences, padded_course_sequences),axis=2)

In [38]:
hooray.shape

(1455, 45, 2)

In [39]:
target = np.array(target)

In [40]:
target.shape

(1455,)

In [43]:

#Two sequential, label encoded features
X = hooray
Y = target

#X_train, X_test, y_train, y_test = train_test_split(X,Y,random_state=2021)

###Define model###
inp = layers.Input((45,2))
feature1 = layers.Lambda(lambda x: x[:,:,0])(inp)
feature2 = layers.Lambda(lambda x: x[:,:,1])(inp)

#Append embeddings features
x1 = layers.Embedding(1445, 5)(feature1)
x2 = layers.Embedding(1445, 7)(feature2)

#LSTMs
feature1 = layers.LSTM(8, return_sequences=True)(x1)
feature1 = layers.LSTM(8)(feature1)

feature2 = layers.LSTM(8, return_sequences=True)(x2)
feature2 = layers.LSTM(8)(feature2)

#Combine LSTM final states
x = layers.concatenate([x1,x2])
out = layers.Dense(1)(x)

model = Model(inp, out)
utils.plot_model(model, show_layer_names=False, show_shapes=True)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [44]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])
model.fit(X,Y, epochs=30, validation_split = .2)

Epoch 1/30
37/37 [==============================] - 0s 4ms/step - loss: 2.4512 - accuracy: 0.3359 - val_loss: 1.2990 - val_accuracy: 0.4399
Epoch 2/30
37/37 [==============================] - 0s 1ms/step - loss: 1.3834 - accuracy: 0.3359 - val_loss: 1.0970 - val_accuracy: 0.4399
Epoch 3/30
37/37 [==============================] - 0s 2ms/step - loss: 1.2001 - accuracy: 0.3359 - val_loss: 0.9833 - val_accuracy: 0.4399
Epoch 4/30
37/37 [==============================] - 0s 1ms/step - loss: 1.0696 - accuracy: 0.3359 - val_loss: 0.8950 - val_accuracy: 0.4399
Epoch 5/30
37/37 [==============================] - 0s 2ms/step - loss: 0.9621 - accuracy: 0.3391 - val_loss: 0.8203 - val_accuracy: 0.4580
Epoch 6/30
37/37 [==============================] - 0s 2ms/step - loss: 0.8695 - accuracy: 0.3666 - val_loss: 0.7581 - val_accuracy: 0.4874
Epoch 7/30
37/37 [==============================] - 0s 2ms/step - loss: 0.7905 - accuracy: 0.4266 - val_loss: 0.7046 - val_accuracy: 0.5371
Epoch 8/30
37/37 [==

In [45]:
model.history.history

{'loss': [2.4512054920196533,
  1.3833792209625244,
  1.2001181840896606,
  1.069641351699829,
  0.9620771408081055,
  0.8695030212402344,
  0.7905457615852356,
  0.7213184833526611,
  0.6628780961036682,
  0.6141045093536377,
  0.5751020908355713,
  0.5466899275779724,
  0.5278627872467041,
  0.513586163520813,
  0.5014046430587769,
  0.49311545491218567,
  0.4857247471809387,
  0.4791771471500397,
  0.474350243806839,
  0.4699798822402954,
  0.46653714776039124,
  0.462510347366333,
  0.4597660303115845,
  0.4570811688899994,
  0.45533332228660583,
  0.45255669951438904,
  0.4503863751888275,
  0.4492607116699219,
  0.44767898321151733,
  0.4462898373603821],
 'accuracy': [0.3359106481075287,
  0.3359106481075287,
  0.3359106481075287,
  0.3359106481075287,
  0.3390989601612091,
  0.3666093945503235,
  0.42657506465911865,
  0.5241504907608032,
  0.5983772277832031,
  0.6660557985305786,
  0.7077891826629639,
  0.7392132878303528,
  0.7603474855422974,
  0.7758303880691528,
  0.78442

In [50]:
target = []
for s_id in student_ids:
    grad = df[df['alt_id'] == s_id].iloc[0].Graduated
    target.append(grad)
    
student_ids = df.alt_id.unique()
course_sequences = []
for s_id in student_ids:
    frame = df[df['alt_id'] == s_id]
    term_sequences = []
    for idx, val in enumerate(frame['Sttr Term'].unique()):
        term = frame[frame['Sttr Term'] == val].sort_values('course_title')
        course_group = term.course_title.str.replace(' ', '').str.cat(sep='')
        term_sequences.append(course_group)
    term_sequences = ' '.join(term_sequences)
    course_sequences.append(term_sequences)

In [47]:
term_sequences

'IntrotoMediaStudiesMusicinFilmSemCriticalThinkingWriting ClassVoiceFilmProductionIIntroductiontoFictionIntroductiontoPsychology'

In [48]:
course_sequences[0]

'IntermediateWritingIntrotoAmerGovt&PolOrganismalBiology ExplorationProject HistoryofAmericanPeopleIIIntroductiontoPsychologyOrganismalBiologyPhilosophyofReligionSkillsinChemistry IntermediateWritingIntroGlobalCompStudiesLogicWhyPeopBelieveWeirdThings ContemporaryPhilosophyIntroductiontoSociologyWhyWeCareAboutthePuritans IntroEnvironmentalStudies EuropeanCivilII(IA)HistofWesternPhilosophyILogicMidEastin20thCent(IA) ExplorationProject EnvironmentalProbandPolicyPrinofMicroeconomicsReligionandtheNaturalWorldUrbanSociology Two-DimensionalDesignUESInternship AsianPersponEnvironScienceEvolutionaryEcologyModernPhilosophyR3urbanPlan,Develop&Design EnvironmentalEarthSciencesEnvironmentalFieldMethodsSrSeminarinEnvironStudiesTchgExpinUrbanEnvStudiesTopicsinAmericanPhilosophy'

In [53]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(course_sequences)
encoded_sequences = tokenizer.texts_to_sequences(course_sequences)
padded_course_sequences = pad_sequences(
encoded_sequences, maxlen=45, dtype='int32', padding='post',
truncating='post', value=0.0)
feature_encoder = OneHotEncoder(handle_unknown='ignore')
feature_ids = list(tokenizer.word_index.values())
feature_encoder.fit(np.array(feature_ids).reshape(-1,1))
feature_encoder.transform(np.array(padded_course_sequences[0]).reshape(-1,1)).reshape(-1,1)
padded_reshaped_sequences = padded_course_sequences.reshape(1455, 45, 1)

model = Sequential()
input_layer = Input(shape=(80, 1))
sequential_1 = LSTM(64, activation='relu')
output_layer = Dense(1, activation='sigmoid')

model.add(input_layer)
model.add(sequential_1)
model.add(output_layer)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])
model.fit(padded_reshaped_sequences, np.array(target), epochs=20, validation_split = .2)


pd.DataFrame(feature_encoder.transform(np.array(padded_reshaped_sequences[0]).reshape(-1,1)).todense())
np.array(padded_course_sequences[0]).reshape(-1,1)

Epoch 1/20
37/37 [==============================] - 1s 28ms/step - loss: 102.1583 - accuracy: 0.5704 - val_loss: 289.9015 - val_accuracy: 0.5052
Epoch 2/20
37/37 [==============================] - 1s 18ms/step - loss: 190.8974 - accuracy: 0.5077 - val_loss: 151.0368 - val_accuracy: 0.5636
Epoch 3/20
37/37 [==============================] - 1s 16ms/step - loss: 156.4861 - accuracy: 0.5275 - val_loss: 440.9733 - val_accuracy: 0.5052
Epoch 4/20
37/37 [==============================] - 1s 15ms/step - loss: 286.0645 - accuracy: 0.4871 - val_loss: 314.6613 - val_accuracy: 0.5086
Epoch 5/20
37/37 [==============================] - 0s 12ms/step - loss: 164.1468 - accuracy: 0.5619 - val_loss: 272.7901 - val_accuracy: 0.5533
Epoch 6/20
37/37 [==============================] - 0s 13ms/step - loss: 206.4371 - accuracy: 0.6186 - val_loss: 536.5527 - val_accuracy: 0.6220
Epoch 7/20
37/37 [==============================] - 0s 12ms/step - loss: 255.5276 - accuracy: 0.6572 - val_loss: 714.4623 - val_ac

array([[360],
       [979],
       [  1],
       [980],
       [981],
       [982],
       [ 39],
       [ 17],
       [  2],
       [983],
       [  2],
       [  1],
       [984],
       [ 97],
       [985],
       [986],
       [ 66],
       [120],
       [987],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0]], dtype=int32)

In [ ]:
def preprocessing_courses(data):
    data.drop(data.index[51518:51537], inplace=True)
    data = data.loc[(data['Cohort'] == '14/FA') | (data['Cohort'] == '15/FA') | (data['Cohort'] == '16/FA')
              | (data['Cohort'] == '17/FA')]
    data = data.dropna(subset=['Sttr Term'])
    mapping = {'N': 0, 'Y': 1}
    data['Graduated'] = data['Graduated'].map(mapping)
    y = data['Graduated']
    X = data['course_title']
    data['alt_id'] = data['alt_id'].astype(int)
    df = data.sort_values(['Sttr Term'], ascending=True)

In [ ]:
df.drop(df.index[51518:51537], inplace=True)
df = df.loc[(df['Cohort'] == '14/FA') | (df['Cohort'] == '15/FA') | (df['Cohort'] == '16/FA')
              | (df['Cohort'] == '17/FA')]
df = df.dropna(subset=['Sttr Term'])
#mapping = {'N': 0, 'Y': 1}
#df['Graduated'] = df['Graduated'].map(mapping)
#y = df['Graduated']
#X = df['course_title']
df['alt_id'] = df['alt_id'].astype(int)
df = df.sort_values(['Sttr Term'], ascending=True)

In [ ]:
newdf.info()

In [ ]:
#regulating Course Sequence
target = []
for s_id in student_ids:
    grad = df[df['alt_id'] == s_id].iloc[0].Graduated
    target.append(grad)
    
student_ids = df.alt_id.unique()
course_sequences = []
for s_id in student_ids:
    frame = df[df['alt_id'] == s_id]
    course_ids = ' '.join(frame.course_title.str.replace(' ', '').to_list())
    course_sequences.append(course_ids)
tokenizer = Tokenizer(lower=False)
tokenizer.fit_on_texts(course_sequences)

encoded_sequences = tokenizer.texts_to_sequences(course_sequences)
padded_course_sequences = pad_sequences(encoded_sequences, maxlen=45, dtype='int32', padding='post',
truncating='post', value=0.0)
feature_encoder = OneHotEncoder(handle_unknown='ignore')
feature_ids = list(tokenizer.word_index.values())
feature_encoder.fit(np.array(feature_ids).reshape(-1,1))
feature_encoder.transform(np.array(padded_course_sequences[0]).reshape(-1,1)).reshape(-1,1)
padded_reshaped_courses = padded_course_sequences.reshape(1455, 45, 1)

X = padded_reshaped_courses
y = target
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2021)
X_train_final, X_val, y_train_final, y_val = train_test_split(X_train, y_train, random_state=2021)


X_train_tokens = (X_train_final)
X_val_tokens = (X_val)
X_test_tokens = (X_test)

lb = LabelBinarizer()
lb.fit(y_train_final)
y_train_lb = to_categorical(lb.transform(y_train_final))[:, 1]
y_val_lb = to_categorical(lb.transform(y_val))[:, 1]
y_test_lb = to_categorical(lb.transform(y_test))[:, 1]

baseline_model = Sequential()
input_layer = Input(shape=(80, 1))
sequential_1 = LSTM(64, activation='relu')
output_layer = Dense(1, activation='sigmoid')

baseline_model.add(input_layer)
baseline_model.add(sequential_1)
baseline_model.add(output_layer)

baseline_model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])
baseline_model_val = baseline_model.fit(X_train_tokens, y_train_lb, epochs=150, batch_size=250, validation_data = (X_val_tokens, y_val_lb))

#pd.DataFrame(feature_encoder.transform(np.array(padded_course_sequences[0]).reshape(-1,1)).todense())
#np.array(padded_course_sequences[0]).reshape(-1,1)

In [76]:
target = []
for s_id in student_ids:
    grad = df[df['alt_id'] == s_id].iloc[0].Graduated
    target.append(grad)

In [77]:
#Sophomore Baseline Grades
student_ids = df.alt_id.unique()
grade_sequences = []
for s_id in student_ids:
    frame = df[df['alt_id'] == s_id]
    grades = ' '.join(frame['Sttr Stu Final Grades'].to_list())
    grade_sequences.append(grades)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(grade_sequences)
encoded_sequences = tokenizer.texts_to_sequences(grade_sequences)
padded_grade_sequences = pad_sequences(
encoded_sequences, maxlen=16, dtype='int32', padding='post',
truncating='post', value=0.0)
feature_encoder = OneHotEncoder(handle_unknown='ignore')
feature_ids = list(tokenizer.word_index.values())
feature_encoder.fit(np.array(feature_ids).reshape(-1,1))
feature_encoder.transform(np.array(padded_grade_sequences[0]).reshape(-1,1)).reshape(-1,1)
padded_reshaped_grades = padded_grade_sequences.reshape(1455, 16, 1)

model = Sequential()
input_layer = Input(shape=(1455, 1))
sequential_1 = LSTM(64, activation='relu')
output_layer = Dense(1, activation='sigmoid')

model.add(input_layer)
model.add(sequential_1)
model.add(output_layer)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])
model.fit(padded_reshaped_grades, np.array(target), epochs=10, validation_split = .2)


#pd.DataFrame(feature_encoder.transform(np.array(padded_grade_sequences[0]).reshape(-1,1)).todense())
#np.array(padded_grade_sequences[0]).reshape(-1,1)

Epoch 1/10
37/37 [==============================] - 0s 10ms/step - loss: 0.5692 - accuracy: 0.8162 - val_loss: 0.6092 - val_accuracy: 0.7766
Epoch 2/10
37/37 [==============================] - 0s 5ms/step - loss: 0.4949 - accuracy: 0.8565 - val_loss: 0.5242 - val_accuracy: 0.7869
Epoch 3/10
37/37 [==============================] - 0s 5ms/step - loss: 0.4266 - accuracy: 0.8548 - val_loss: 0.4695 - val_accuracy: 0.7904
Epoch 4/10
37/37 [==============================] - 0s 5ms/step - loss: 0.3866 - accuracy: 0.8565 - val_loss: 0.4179 - val_accuracy: 0.8041
Epoch 5/10
37/37 [==============================] - 0s 5ms/step - loss: 0.3599 - accuracy: 0.8634 - val_loss: 0.4278 - val_accuracy: 0.8179
Epoch 6/10
37/37 [==============================] - 0s 5ms/step - loss: 0.3334 - accuracy: 0.8746 - val_loss: 0.4624 - val_accuracy: 0.8247
Epoch 7/10
37/37 [==============================] - 0s 6ms/step - loss: 0.3357 - accuracy: 0.8746 - val_loss: 0.3898 - val_accuracy: 0.8351
Epoch 8/10
37/37 [=

In [72]:
target = []
for s_id in student_ids:
    grad = df[df['alt_id'] == s_id].iloc[0].Graduated
    target.append(grad)

In [81]:
#Sophomore Course Sequence
student_ids = df.alt_id.unique()
course_sequences = []
for s_id in student_ids:
    frame = df[df['alt_id'] == s_id]
    course_ids = ' '.join(frame.course_title.str.replace(' ', '').to_list())
    course_sequences.append(course_ids)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(course_sequences)
encoded_sequences = tokenizer.texts_to_sequences(course_sequences)
padded_course_sequences = pad_sequences(encoded_sequences, maxlen=16, dtype='int32', padding='post',
truncating='post', value=0.0)
feature_encoder = OneHotEncoder(handle_unknown='ignore')
feature_ids = list(tokenizer.word_index.values())
feature_encoder.fit(np.array(feature_ids).reshape(-1,1))
feature_encoder.transform(np.array(padded_course_sequences[0]).reshape(-1,1)).reshape(-1,1)
padded_reshaped_courses = padded_course_sequences.reshape(1455, 16, 1)

model = Sequential()
input_layer = Input(shape=(1455, 1))
sequential_1 = LSTM(64, activation='relu')
output_layer = Dense(1, activation='sigmoid')

model.add(input_layer)
model.add(sequential_1)
model.add(output_layer)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])
model.fit(padded_reshaped_courses, np.array(target), epochs=10, validation_split = .2)


pd.DataFrame(feature_encoder.transform(np.array(padded_course_sequences[0]).reshape(-1,1)).todense())
np.array(padded_course_sequences[0]).reshape(-1,1)

Epoch 1/10
37/37 [==============================] - 0s 7ms/step - loss: 1.4563 - accuracy: 0.5636 - val_loss: 0.8805 - val_accuracy: 0.5361
Epoch 2/10
37/37 [==============================] - 0s 3ms/step - loss: 0.7974 - accuracy: 0.6100 - val_loss: 0.7211 - val_accuracy: 0.5533
Epoch 3/10
37/37 [==============================] - 0s 3ms/step - loss: 0.7119 - accuracy: 0.6271 - val_loss: 0.7079 - val_accuracy: 0.5326
Epoch 4/10
37/37 [==============================] - 0s 3ms/step - loss: 0.6610 - accuracy: 0.6469 - val_loss: 0.6912 - val_accuracy: 0.5601
Epoch 5/10
37/37 [==============================] - 0s 3ms/step - loss: 0.6472 - accuracy: 0.6607 - val_loss: 0.6827 - val_accuracy: 0.5498
Epoch 6/10
37/37 [==============================] - 0s 3ms/step - loss: 0.6438 - accuracy: 0.6529 - val_loss: 0.6832 - val_accuracy: 0.5704
Epoch 7/10
37/37 [==============================] - 0s 3ms/step - loss: 0.6215 - accuracy: 0.6718 - val_loss: 0.6787 - val_accuracy: 0.6082
Epoch 8/10
37/37 [==

array([[40],
       [13],
       [12],
       [10]], dtype=int32)